In [1]:
from urllib.request import urlopen
from urllib import parse
from bs4 import BeautifulSoup
from Models.DataCollector import DataCollector
import json

In [2]:
class UserInspector(DataCollector):
    def __init__(self):
        self.name_to_key = {}
        self.name_to_id = {}
        self.id_to_key = {}
        self.key_to_id = {}
        with open('./metadata/name_to_key.json', 'r') as fp:
            self.name_to_key = json.load(fp)

        with open('./metadata/name_to_id.json', 'r') as fp:
            self.name_to_id = json.load(fp)

        with open('./metadata/id_to_key.json', 'r') as fp:
            self.id_to_key = json.load(fp)

        with open('./metadata/key_to_id.json', 'r') as fp:
            self.key_to_id = json.load(fp)

    def user_history_collector(self, userName):
        # url encoding (for Korean words)
        userName = parse.quote(userName)

        # open url and create bs4 Object
        html = urlopen("https://www.op.gg/summoner/userName="+userName)
        bsObject = BeautifulSoup(html, "html.parser")

        # decode userName from url encoding
        userName = parse.unquote(userName)

        # win/loss and play count
        wins = int(bsObject.find('span', 'wins').text[:-1])
        losses = int(bsObject.find('span', 'losses').text[:-1])
        total_play = wins + losses    
        win_rate = round(wins/total_play, 3)

        champ_list = bsObject.find_all("div", {'class' : 'ChampionBox Ranked'})
        champion_history = self.champ_history_setter(champ_list)            

        # this is (adapted) user_history DTO
        user_historyDTO = {
            'user_name' : userName,
            'total_play' : total_play,
            'win_rate' : win_rate,
            'champion_history' : champion_history,
        }
        return user_historyDTO

In [3]:
import torch

from Models.Models import AutoEncoder, Predictor
from utils import champ_id_remap, global_win_rate

In [7]:
class ChampionRecommender():
    def __init__(self):
        self.user_inspector = UserInspector()

        self.user_encoder = AutoEncoder(143, 12)
        self.user_encoder.load_state_dict(torch.load('./trained_model/user_encoder_augmented.pth'))

        self.item_encoder = AutoEncoder(143, 10)
        self.item_encoder.load_state_dict(torch.load('./trained_model/item_encoder.pth'))

        self.predictor = Predictor(user_len=12, item_len=10, hidden_unit=22)
        self.predictor.load_state_dict(torch.load('./trained_model/predictor_augmented.pth'))
        
        self.remapped_champ_id = champ_id_remap()
        self.global_win_rate = global_win_rate() 
        
        with open('./datasets/item_vectors_tf_idf.json', 'r') as fp:
            self.item_vectors = json.load(fp)        
        
        with open('./metadata/user_idf_table.json', 'r') as fp:
            self.idf_table = json.load(fp)

    def _recommender(self, userName):
        #try:
        user_data = self.user_inspector.user_history_collector(userName)
        win_rate_dict = self._winrate_predictior(user_data)
        return win_rate_dict
        #except:
        #    print("not a valid user. please check.")


    def _winrate_predictior(self, user_data):
        user_winrate = self._tensor_item(user_data['win_rate'])
        user_vec = self._user_vector_generator(user_data)
        played_champions = [i for i, e in enumerate(user_vec) if e != 0]
        user_vec = torch.Tensor(user_vec)
        user_vec = self.user_encoder.encoder(user_vec)
        
        win_rate_dict = dict()
        
        for key, item in self.item_vectors.items():
            if key not in played_champions:
                global_win = self._tensor_item(self.global_win_rate[int(key)])
                item_vec = torch.Tensor([item])
                item_vec = self.item_encoder.encoder(item_vec)
                item_vec = item_vec.squeeze()
                prediction = self.predictor(user_vec, item_vec, user_winrate, global_win)
                win_rate_dict[key] = prediction
        
        return win_rate_dict
            
            
    def _user_vector_generator(self, user_data):
        user_win_rate = user_data['win_rate']
        play_count_vector = [0 for x in range(143)]

        for champ_history in user_data['champion_history']:
            original_champ_id = champ_history['champion_key']
            champ_idx = self.remapped_champ_id[original_champ_id]

            max_play_count = self._get_max_play_count(user_data)
            measured_score = (champ_history['play_count'] / max_play_count) * self.idf_table[str(champ_idx)]
            play_count_vector[champ_idx] = measured_score
        
        return play_count_vector

    
    def _get_max_play_count(self, user_data):
        """
        Get count of maximally played champion for user_name
        """
        max_play_count = 0

        for champ_history in user_data['champion_history']:
            champ_play_count = champ_history['play_count']
            if max_play_count < champ_play_count:
                max_play_count = champ_play_count

        return max_play_count

    
    def _tensor_item(self, item):
        return torch.Tensor([item])

In [8]:
test = ChampionRecommender()

In [9]:
test._recommender('만년동핵폭탄')

{'0': tensor([0.5401], grad_fn=<AddBackward0>),
 '1': tensor([0.5348], grad_fn=<AddBackward0>),
 '2': tensor([0.5369], grad_fn=<AddBackward0>),
 '3': tensor([0.5368], grad_fn=<AddBackward0>),
 '4': tensor([0.5347], grad_fn=<AddBackward0>),
 '5': tensor([0.5367], grad_fn=<AddBackward0>),
 '6': tensor([0.5368], grad_fn=<AddBackward0>),
 '7': tensor([0.5381], grad_fn=<AddBackward0>),
 '8': tensor([0.5401], grad_fn=<AddBackward0>),
 '9': tensor([0.5401], grad_fn=<AddBackward0>),
 '10': tensor([0.5368], grad_fn=<AddBackward0>),
 '11': tensor([0.5366], grad_fn=<AddBackward0>),
 '12': tensor([0.5382], grad_fn=<AddBackward0>),
 '13': tensor([0.5382], grad_fn=<AddBackward0>),
 '14': tensor([0.5401], grad_fn=<AddBackward0>),
 '15': tensor([0.5382], grad_fn=<AddBackward0>),
 '16': tensor([0.5347], grad_fn=<AddBackward0>),
 '17': tensor([0.5367], grad_fn=<AddBackward0>),
 '18': tensor([0.5346], grad_fn=<AddBackward0>),
 '19': tensor([0.5368], grad_fn=<AddBackward0>),
 '20': tensor([0.5369], grad_f

In [8]:
a = torch.randn(3, 1)

In [9]:
len(a.shape) == 1 and len(a.shape)

True